In [ ]:
import sys
sys.path.append('..')

In [ ]:
from pathlib import Path

In [ ]:
import torch
import torch.nn as nn

In [ ]:
from torch.autograd import Variable

In [ ]:
from models.ssd import SSD
from models.ssd.detector import Detector

In [ ]:
MODEL_FILE = '/home/jiun/datasets/weights/cylinder/ssd-mb2-no-sched/SSD300-100000.pth'

In [ ]:
num_classes = 2
batch = 1

In [ ]:
model = SSD.__new__(SSD, num_classes, batch)

In [ ]:
model.load(torch.load(MODEL_FILE, map_location=lambda s, l: s))
model.eval()

In [ ]:
Detector.init(num_classes, batch, nms=False)

In [ ]:
inputs = torch.ones(1, 3, 300, 300)
output = model(inputs)

In [ ]:
print(output.min(), output.max())
print(output.mean(), output.std())

In [ ]:
inputs = torch.FloatTensor(1, 3, 300, 300)
_ = model(inputs)

module = torch.jit.trace(model, inputs)
torch.jit.save(module, 'amano-script.pt')

In [ ]:
show(img.astype(np.uint8))

In [ ]:
from PIL import Image
from IPython.display import display

def show(ary):
    display(Image.fromarray(ary))